In [7]:
from datahelpers.constants import iden, ye, ai, ps, up, dn, ar, ni, cexp, qcexp, nw, wi, dist, rdist, pm, \
                                    cpop, cden, ct, affs, aus
from os.path import expanduser, join
import pandas as pd
from bm_support.add_features import generate_feature_groups
from bm_support.add_features import normalize_columns
from bm_support.supervised_aux import study_sample, metric_selector
from functools import partial
from multiprocessing import Pool
from copy import deepcopy
import warnings
from numpy.random import RandomState
import gzip
import pickle
import argparse

%load_ext autoreload
%autoreload 1
%aimport bm_support.add_features
%aimport bm_support.supervised_aux
%aimport bm_support.supervised

warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
an_version = 12

# origin = 'litgw'
# version = 1

# origin = 'lit'
# version = 8

origin = 'gw'
version = 11

# model_type = 'rf'
# seed0 = 17
n_trials = 1
n_subtrials = 4
n_estimators = 17
# n_estimators = 55
datapath = None
# seed0 = 13
seed0 = 1
n_jobs = 1
verbose = True
model_type = 'lr'

In [9]:
min_log_alpha = -1
max_log_alpha = 1
log_reg_dict = {'min_log_alpha': min_log_alpha, 'max_log_alpha': max_log_alpha}

eps = 0.2
upper_exp, lower_exp = 1 - eps, eps
# thrs = [-1e-8, lower_exp, upper_exp, 1.0001e0]
if datapath:
    col_families = generate_feature_groups(expanduser(join(datapath, 'v12_columns.txt')))
else:
    col_families = generate_feature_groups(expanduser('~/data/kl/columns/v12_columns.txt'))

if verbose:
    print('Number of col families: {0}. Keys: {1}'.format(len(col_families), sorted(col_families.keys())))

col_families = {k: v for k, v in col_families.items() if 'future' not in k}
if verbose:
    print('Number of col families (excl. future): {0}. Keys: {1}'.format(len(col_families),
                                                                         sorted(col_families.keys())))

columns_interest = [x for sublist in col_families.values() for x in sublist]
if datapath:
    df_path = expanduser(join(datapath, '{0}_{1}_{2}.h5'.format(origin, version, an_version)))
else:
    df_path = expanduser('~/data/kl/final/{0}_{1}_{2}.h5'.format(origin, version, an_version))
df = pd.read_hdf(df_path, key='df')

['cpop', 'cden', 'ksst', 'nhi', 'affiliations_affind', 'authors_affind', 'future_affind', 'past_affind', 'affiliations_suppind', 'authors_suppind', 'future_suppind', 'past_suppind', 'affiliations_comm_size', 'authors_comm_size', 'future_comm_size', 'past_comm_size', 'affiliations_ncomms', 'authors_ncomms', 'future_ncomms', 'past_ncomms', 'affiliations_ncomponents', 'authors_ncomponents', 'future_ncomponents', 'past_ncomponents', 'affiliations_size_ulist', 'authors_size_ulist', 'future_size_ulist', 'past_size_ulist', 'pre_authors', 'pre_affs']
{'cpop': 8, 'cden': 8, 'ksst': 8, 'nhi': 2, 'affiliations_affind': 4, 'authors_affind': 4, 'future_affind': 4, 'past_affind': 4, 'affiliations_suppind': 4, 'authors_suppind': 4, 'future_suppind': 4, 'past_suppind': 4, 'affiliations_comm_size': 4, 'authors_comm_size': 4, 'future_comm_size': 4, 'past_comm_size': 4, 'affiliations_ncomms': 4, 'authors_ncomms': 4, 'future_ncomms': 4, 'past_ncomms': 4, 'affiliations_ncomponents': 4, 'authors_ncomponents

In [16]:
# mask: literome - mask out a specific interaction
mask_lit = (df[up] == 7157) & (df[dn] == 1026)

# mask:  interaction with more than 3 claims
thr = 3
mask_len_ = (df.groupby(ni).apply(lambda x: x.shape[0]) > thr)
mask_max_len = df[ni].isin(mask_len_[mask_len_].index)
print(mask_max_len.shape[0], sum(mask_max_len))

# mask : interactions which are between
eps_window_mean = 0.1
mean_col = 0.5
mask_exp = ((df[cexp] <= lower_exp - eps_window_mean) | (df[cexp] >= upper_exp + eps_window_mean)
            # | ((df2[cexp] <= (mean_col + eps_window_mean)) & (df2[cexp] >= (mean_col - eps_window_mean)))
            )

feature_dict = deepcopy(col_families)
# families = ['affiliations_affind', 'affiliations_comm_size', 'affiliations_ncomms',
#             'affiliations_ncomponents', 'affiliations_size_ulist', 'affiliations_suppind',
#             'ai', 'ar', 'authors_affind', 'authors_comm_size', 'authors_ncomms', 'authors_ncomponents',
#             'authors_size_ulist', 'authors_suppind', 'cden', 'citations',
#             'cite_count', 'cpop', 'delta_year', 'ksst', 'lincscomm_size', 'lincssame_comm',
#             'litgwcsize_dn', 'litgwcsize_up',
#             'litgweff_comm_size', 'litgwsame_comm', 'nhi', 'past_affind',
#             'past_comm_size', 'past_ncomms', 'past_ncomponents',
#             'past_size_ulist', 'past_suppind', 'pre_affs', 'pre_authors', 'time']
families = ['affiliations_comm_size',
            'ai', 'ar', 'cden', 'citations',
            'cite_count', 'cpop', 'delta_year', 'ksst', 'lincscomm_size', 'lincssame_comm',
            'litgweff_comm_size', 'litgwsame_comm', 'nhi', 'past_affind',
            'past_comm_size', 'time']

feature_dict = {k: v for k, v in feature_dict.items() if k in families}

trial_features = [x for sublist in feature_dict.values() for x in sublist]

feature_dict_inv = {}
for k, v in feature_dict.items():
    feature_dict_inv.update({x: k for x in v})

# mask: not nulls in trial features
masks = []
for c in trial_features:
    masks.append(df[c].notnull())

mask_notnull = masks[0]
for m in masks[1:]:
    mask_notnull &= m

print('Experimental mask len {0}'.format(sum(mask_exp)))
print('Number of trial features: {0}'.format(len(trial_features)))
print('Number of notnull entries (over all features): {0} from {1}'.format(sum(mask_notnull), mask_notnull.shape))

if origin == 'lit':
    mask_agg = mask_notnull & ~mask_lit
else:
    mask_agg = mask_notnull

dfw = df.loc[mask_agg].copy()

#metric to optimize for
mm = 'precision'

nmax = 5000

rns = RandomState(seed0)
seeds = rns.randint(nmax, size=n_trials)

if model_type == 'lr':
    dfw = normalize_columns(dfw, trial_features)

func = partial(study_sample, dfw=dfw, dist=dist, feature_dict=feature_dict, metric_mode=mm,
               model_type=model_type,
               n_subtrials=n_subtrials, n_estimators=n_estimators,
               log_reg_dict=log_reg_dict, verbose=verbose)

print('Starting {0} processes...'.format(n_jobs))

# lr seems to run 4x
if n_jobs > 1:
    with Pool(n_jobs) as p:
        meta_report = p.map(func, seeds)
else:
    meta_report = list(map(func, seeds))

# if datapath:
#     fout = expanduser(join(datapath, '{0}_{1}_{2}_{3}_seed0_{4}.report.pgz'.format(origin, version,
#                                                                                    an_version, model_type, seed0)))
# else:
#     fout = expanduser('~/data/kl/reports/{0}_{1}_{2}_{3}_seed0_{4}.report.pgz'.format(origin, version,
#                                                                                       an_version,
#                                                                                       model_type, seed0))

# with gzip.open(fout, 'wb') as fp:
#     pickle.dump(meta_report, fp)

15494 7878
Experimental mask len 887
Number of trial features: 90
Number of notnull entries (over all features): 14943 from (15494,)
Starting 1 processes...
*** df_train dist vc
1.0    7944
0.0     826
2.0     195
Name: guess, dtype: int64
model_dict {'n_jobs': 1, 'C': 0.1}
nf: 1 cfeature: litgw_comm_im_dir_wei_pNone... metric: 0.341 metric_improv: 100.00 %
nf: 2 cfeature: litgw_comm_im_undir_unwei_p... metric: 0.385 metric_improv: 11.25 %
nf: 3 cfeature: lincs_comm_ml_undir_unwei_p... metric: 0.415 metric_improv: 7.40 %
nf: 4 cfeature: ksst2...                       metric: 0.426 metric_improv: 2.61 %
nf: 5 cfeature: ksst3...                       metric: 0.523 metric_improv: 18.43 %
nf: 6 cfeature: affiliations_comm_size1...     metric: 0.529 metric_improv: 1.25 %
nf: 7 cfeature: mu...                          metric: 0.585 metric_improv: 9.50 %
nf: 8 cfeature: cpop2...                       metric: 0.585 metric_improv: 0.08 %
nf: 9 cfeature: cden2...                       metric: 0.

In [ ]:
meta_report[0][0]

In [ ]:
report['corr'] = np.corrcoef(y_pred, y_test)[0, 1]
report['accuracy'] = {}
report['accuracy']['normal'] = accuracy_score(y_test, y_pred)
report['accuracy']['balanced'] = accuracy_score(y_test, y_pred)

report['vector'] = {}
report['vector']['precision'] = precision_score(y_test, y_pred, average=mode_scores)
report['vector']['recall'] = recall_score(y_test, y_pred, average=mode_scores)
report['vector']['f1'] = f1_score(y_test, y_pred, average=mode_scores)

report['macro'] = {}
report['macro']['precision'] = precision_score(y_test, y_pred, average='macro')
report['macro']['recall'] = recall_score(y_test, y_pred, average='macro')
report['macro']['f1'] = f1_score(y_test, y_pred, average='macro')

nclasses = report['vector']['precision'] .shape[0]

report['exponent'] = {}
report['exponent'] = {k: np.sum(v ** metric_uniform_exponent)/nclasses for k, v in report['vector']}